# Cat and Dog

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import zipfile
import torch.nn as nn
import torch
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from torchvision import models
from PIL import Image
from collections import OrderedDict
from tqdm import tqdm
from sklearn import model_selection
from statistics import mode

## Define some variables

In [12]:
base_dir = "E:/data/DogCat/dogs-vs-cats-redux-kernels-edition"
train_ = os.path.join(base_dir,'train')
test_ = os.path.join(base_dir,'test')

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
epochs = 20
lr = 0.1
batch_size = 64
corss_val_split = 5
img_size = 128


In [13]:
model = models.resnet18(pretrained=True)

## Show sample DataFrame

In [14]:
sample_df = pd.read_csv(os.path.join(base_dir,'sample_submission.csv'))
sample_df.head()

,id,label
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5


## Creating Dataframes for Training

In [15]:
train_dir = './train/train'
test_dir = './test/test'

train_df = pd.DataFrame()
test_df = pd.DataFrame()
train_df['file_path'] = glob.glob(r'E:/data/DogCat/dogs-vs-cats-redux-kernels-edition/train/train/*.*')
train_df['file_name'] = [os.path.basename(val) for val in train_df['file_path']]
train_df['file_id'] = train_df['file_name'].str.split('.').str[1]
train_df['label'] = train_df['file_name'].str.split('.').str[0]
train_df['label'] = train_df['label'].map({'cat':0,'dog':1})


test_df['file_path'] = glob.glob(r'E:/data/DogCat/dogs-vs-cats-redux-kernels-edition/test/test/*.*')
test_df['file_name'] = [os.path.basename(val) for val in test_df['file_path']]
test_df['file_id'] = test_df['file_name'].str.split('.').str[0]
test_df['label'] = 9999

In [17]:
train_df.head()

,file_path,file_name,file_id,label,kfold
0,E:/data/DogCat/dogs-vs-cats-redux-kernels-edit...,cat.0.jpg,0,0,0
1,E:/data/DogCat/dogs-vs-cats-redux-kernels-edit...,cat.1.jpg,1,0,4
2,E:/data/DogCat/dogs-vs-cats-redux-kernels-edit...,cat.10.jpg,10,0,3
3,E:/data/DogCat/dogs-vs-cats-redux-kernels-edit...,cat.100.jpg,100,0,1
4,E:/data/DogCat/dogs-vs-cats-redux-kernels-edit...,cat.1000.jpg,1000,0,4


## Creating Folds for train data

In [18]:
train_df['kfold'] = -1
y = train_df.label.values

kf = model_selection.StratifiedKFold(n_splits=corss_val_split, shuffle=True, random_state=42)

for f,(t_idx,val_idx) in enumerate(kf.split(X=train_df, y=y)):
    train_df.loc[val_idx, 'kfold'] = f
    
train_df.to_csv("train_folds.csv", index=False)

In [19]:
train_df = pd.read_csv("train_folds.csv")
train_df.head()

,file_path,file_name,file_id,label,kfold
0,E:/data/DogCat/dogs-vs-cats-redux-kernels-edit...,cat.0.jpg,0,0,0
1,E:/data/DogCat/dogs-vs-cats-redux-kernels-edit...,cat.1.jpg,1,0,4
2,E:/data/DogCat/dogs-vs-cats-redux-kernels-edit...,cat.10.jpg,10,0,3
3,E:/data/DogCat/dogs-vs-cats-redux-kernels-edit...,cat.100.jpg,100,0,1
4,E:/data/DogCat/dogs-vs-cats-redux-kernels-edit...,cat.1000.jpg,1000,0,4


### Define Dataset Class 

In [20]:
class CatDogset(Dataset):
    def __init__(self,df,mode='train',transform=None):
        self.df = df
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img = Image.open(self.df.loc[:,'file_path'][idx])
        label = self.df.loc[:,'label'][idx]
        
        if self.transform:
            img = self.transform(img)
            
        if self.mode == 'train':
            img = img.numpy()
            return img.astype('float32'), label
        else:
            img = img.numpy()
            return img.astype('float32'),label

## Define Transformation

In [21]:
train_transform = transforms.Compose([transforms.Resize((img_size,img_size)),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
                                     transforms.RandomHorizontalFlip(0.5)])

val_transform = transforms.Compose([transforms.Resize((img_size,img_size)),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

test_transform = transforms.Compose([transforms.Resize((img_size,img_size)),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

## Creating sample loader for visualization

In [22]:
sample_set = CatDogset(train_df,mode='train',transform=train_transform)
sample_loader = DataLoader(sample_set,batch_size=batch_size,shuffle=True,num_workers=4)

In [23]:
def display_batch():
    figure = plt.figure(figsize=(15,15))
    clos, rows = 8,8
    for i in range(1,clos*rows):
        img,label = next(iter(sample_loader))
        figure.add_subplot(rows,clos,i)
        plt.title(label[i])
        plt.axis('off')
        plt.imshow(img[i].permute(1,2,0),camap='gray')
    plt.show()
    
# display_batch()

In [24]:
for param in model.parameters():
    param.requires_grad = False
fc = nn.Sequential(OrderedDict([
    ('fc',nn.Linear(512,100)),
('relu',nn.ReLU()),
('fc2',nn.Linear(100,2)),
('output',nn.LogSoftmax(dim=1))]))

model.fc = fc
model = model.to(device)

In [25]:
optimizer = optim.Adam(model.fc.parameters(),lr=lr)
criterion = nn.NLLLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=5,verbose=True)

In [37]:
def fit(dataset,dataloader,criterion,model,optimizer):
    model.train()
    for bi,(img,lab) in tqdm(enumerate(dataloader),total=int(len(dataset)/dataloader.batch_size)):
        img,lab = img.to(device), lab.to(device)
        optimizer.zero_grad()
        outputs = model(img)
        loss = criterion(outputs,lab)
        loss.backward()
        optimizer.step()

In [41]:
def evaluate(dataset,dataloader,criterion,model):
    model.eval()
    final_loss = 0
    counter = 0
    for bi,(img,lab) in tqdm(enumerate(dataloader),total=int(len(dataset)/dataloader.batch_size)):
        counter += 1
        img,lab = img.to(device), lab.to(device)
        output = model(img)
        loss = criterion(output,lab)
        final_loss += loss
    return final_loss/counter

In [39]:
def train_fold(fold):
    print(f"For Fold={fold}")
    for epoch in range(epochs):
        print(f"Epoch={epoch}")
        train_dataframe = train_df.query('kfold!=@fold').reset_index(drop=True)
        val_dataframe = train_df.query('kfold==@fold').reset_index(drop=True)

        trainset = CatDogset(train_dataframe,mode = 'train' ,transform = train_transform)
        train_loader = DataLoader(trainset, batch_size = batch_size, shuffle=True, num_workers=0)

        valset = CatDogset(val_dataframe,mode = 'test' ,transform = val_transform)
        val_loader = DataLoader(valset,batch_size = batch_size, shuffle=False, num_workers=0)


        fit(trainset,train_loader,criterion,model,optimizer)
        val_score = evaluate(valset,val_loader,criterion,model)
        scheduler.step(val_score)
        torch.save(model.state_dict(),f"resnet18_fold_{fold}.bin")

In [42]:
train_fold(0)
train_fold(1)
train_fold(2)
train_fold(3)
train_fold(4)

For Fold=0
Epoch=0


313it [01:12,  4.34it/s]                                                                                               
79it [00:32,  2.44it/s]                                                                                                


Epoch=1


313it [01:12,  4.29it/s]                                                                                               
79it [00:16,  4.69it/s]                                                                                                


Epoch=2


...

313it [01:19,  3.93it/s]                                                                                               
79it [00:19,  4.05it/s]                                                                                                


Epoch=18


313it [01:24,  3.72it/s]                                                                                               
79it [00:19,  4.14it/s]                                                                                                


Epoch=19


313it [01:24,  3.71it/s]                                                                                               
79it [00:19,  4.11it/s]                                                                                                


In [43]:
def prediction_fold(fold):
    model.load_state_dict(torch.load(f'./resnet18_fold_{fold}.bin'))
    model.eval()
    pred_list = []
    test_dataset = CatDogset(test_df,mode = 'test' ,transform = test_transform)
    testloader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, num_workers=0)
    for x, fn in tqdm(testloader):
        with torch.no_grad():
            x = x.to(device)
            output = model(x)
            pred = torch.argmax(output, dim=1)
            pred_list += [p.item() for p in pred]
    return pred_list

In [44]:
p1 = prediction_fold(0)
p2 = prediction_fold(1)
p3 = prediction_fold(2)
p4 = prediction_fold(3)
p5 = prediction_fold(4)

100%|████████████████████████████████████████████████████████████████████████████████| 196/196 [00:48<00:00,  4.05it/s]


In [45]:
final_predictions = [mode(val) for val in list(zip(p1,p2,p3,p4,p5))]

In [46]:
submission = sample_df[['id']].merge(pd.DataFrame({'id':test_df['file_id'].astype(int),'label':final_predictions}),how='left',on='id')
submission.to_csv('preds_resnet18.csv', index=False)